In [24]:
import torch
import torch.nn as nn

In [45]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, output_size):
    
        super().__init__()
        self.lstm = nn.LSTM(input_size, 128)
        self.linear = nn.Linear(128, output_size)
        
    def reset(self, batch_size):
        zero_state = torch.zeros(1, batch_size, 128)
        self.hidden_state = zero_state
        self.cell_state = zero_state
        
    def logits(self, x):
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        return self.linear(out[-1].reshape(-1, 128))

    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

In [46]:
char_encodings = [
    [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],  # ' '
    [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],  # 'a'
    [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],  # 'c'
    [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],  # 'f'
    [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],  # 'h'
    [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],  # 'l'
    [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],  # 'm'
    [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],  # 'n'
    [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],  # 'o'
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],  # 'p'
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],  # 'r'
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],  # 's'
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]   # 't'
]

encoding_size = len(char_encodings)

index_to_char = [' ', 'a', 'c', 'f', 'h', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't']

In [47]:
 emoji_encodings = [
    [1., 0., 0., 0., 0., 0., 0.], # '🎩'
    [0., 1., 0., 0., 0., 0., 0.], # '🐀'
    [0., 0., 1., 0., 0., 0., 0.], # '🐈'
    [0., 0., 0., 1., 0., 0., 0.], # '🏢'
    [0., 0., 0., 0., 1., 0., 0.], # '🧔'
    [0., 0., 0., 0., 0., 1., 0.], # '🧢'
    [0., 0., 0., 0., 0., 0., 1.]  # '👦'
]

emoji_size = len(emoji_encodings)
index_to_emoji = ['🎩', '🐀', '🐈', '🏢', '🧔', '🧢', '👦']

In [48]:
def encode(string):
    encoding = []
    
    for char in string:
        encoding.append(char_encodings[index_to_char.index(char)])
        
    return encoding

In [49]:
def encode_emoji(emoji):
    return emoji_encodings[index_to_emoji.index(emoji)]

In [50]:
def decode_emoji(tensor):
    return index_to_emoji[tensor.argmax(1)]

In [51]:
x_train = torch.tensor([
    encode('hat '),
    encode('rat '),
    encode('cat '),
    encode('flat'),
    encode('matt'),
    encode('cap '),
    encode('son '),
]).transpose(1, 0)

In [52]:
y_train = torch.tensor([encode_emoji('🎩'), encode_emoji('🐀'), encode_emoji('🐈'), encode_emoji('🏢'), 
                        encode_emoji('🧔'), encode_emoji('🧢'), encode_emoji('👦')]) 

In [53]:
model = LSTM(encoding_size, emoji_size)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [80]:
epochs = 500

for epoch in range(epochs):
    model.reset(x_train.size(1))
    model.loss(x_train, y_train).backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 9:
        model.reset(1)
        test_string = 'rt  '
        print(decode_emoji(model.f(torch.tensor([encode(test_string)]).transpose(1, 0))))

🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
🐀
